# Go to the bottom and read the instruction
# Do not Run the Algorithm

In [22]:
import pandas as pd
import numpy as np
import geopandas as gpd
import math
import geopy

import matplotlib.pyplot as plt
import shapely
%matplotlib inline

from geopy.distance import vincenty
from geopy.distance import great_circle

from shapely.geometry import Point


In [23]:
class MyDf:
    def __init__(self, mydf):
        self.df = mydf

In [24]:
def geome (x,y):
    return Point(x,y)

In [25]:
def point_close_to_station (All,radius,center_X,center_Y,x_colum,y_colum):
    #The x coordinate in df HAVE TO BE on the first column
    #The y coordinate in df HAVE TO BE on the second column
    pointclosetome=gpd.GeoDataFrame()    
    for i in range(0,len(All)):
        if (vincenty((center_Y,center_X),
                   (All[y_colum][i],All[x_colum][i])).meters<=radius^2):
             pointclosetome = pointclosetome.append(All.iloc[i],ignore_index=False)
    return pointclosetome




In [26]:
def population_of_an_area (point_close_to_station,pop_index):
    return point_close_to_station[pop_index].sum()

In [27]:
def turnoff_poin(All_Point,point_close_to_station):
    #Take out from a dataframe the point selected insied the circle of
    #fruiability of the station
    return All_Point.drop(point_close_to_station.index.values)


In [28]:
def compare(A, pop_index):
    tmp=-1
    index=0
    for k in range(len(A)):
        if (A[k].df[pop_index].sum()>tmp):
            tmp=A[k].df[pop_index].sum()
            index=k
    
    return index

In [29]:
def adjustAll (All,i):
    
    arr=range(0,len(All))
    arr.remove(1)
    arr.insert(0,1)
    All=All.reindex(arr)

    return All.reset_index(drop=True)  


In [30]:
def findthebest (All,radius,x_colum,y_colum,pop_index):
    tmp=0
    for k in range(0,len(All)):
        fruiability = point_close_to_station(All,radius,All[x_colum][k],All[y_colum][k],x_colum,y_colum)
        population = population_of_an_area(fruiability,pop_index)
        if population>tmp:
            pnt=k
        

    return k,population,fruiability
                
    

In [45]:
def configuration2 (All,radius,number_of_station,x_colum,y_colum,pop_index):
    
    if number_of_station> len(All):
        print ('ERROR, YOU WANT TO PUT MORE STATION THAN POSSIBLE LOCATION')
        print ('THE MAXIMUM POSSIBLE NUMBER OF STATION IS', len(All))
        print ('THE ALGORITH WILL RESET FOR YOU THE OPTION AND FIND ALL THE BEST')
        print ('CONFIGURATION FOR n STATION WITH n THAT GOES FROM 0 TO', len(All))
        maximum_mumber_of_station=len(All)

    path=pd.DataFrame()

    while ((len(All)>0) and (number_of_station>0)):
            station, popcoverd, fruiability = findthebest (All,radius,x_colum,y_colum,pop_index)
            path = path.append({'x':All[x_colum][station], 'y':All[y_colum][station], 'pop': popcoverd},ignore_index=True)
            All = turnoff_poin(All,fruiability)
            All = All.reset_index(drop=True)
            number_of_station=number_of_station-1
            

    path.to_csv('configuration') 

    return
       


In [32]:
def create_up_to_k_option (All,radius,maximum_mumber_of_station,x_colum,y_colum,pop_index):
   
        
    lis=[]
    a=pd.DataFrame()
    a['number of station']=[]
    a['population covered']=[]
    #a['configuration']=[]
    for k in range(1,maximum_mumber_of_station+1):
        b=MyDf(configuration2(All,radius,k,x_colum,y_colum,pop_index))
        a = a.append({'number of station':len(b.df), 
                      'population covered': population_of_an_area(b.df,'pop')},
                     # 'configuration': b},
                     ignore_index=True)
        if k>2:
            if (a['number of station'][k-1]==a['number of station'][k-2]):
                print ('NO REASON TO PUT', maximum_mumber_of_station, 'STATION') 
                print('ALL THE AREA HAS BEEN COVERED WITH JUST', k-1 ,'STATION')
                print ('THE ALGORITHM STOPS HERE.')
                a.drop([k-1], inplace=True)
                break
        r=str(k)
        b.df.to_csv('station/' + r)
    
    a.to_csv('summary')
    return
        

In [33]:
def printing_the_curve(lis):
    station=[]
    population=[]
    for i in range(len(lis)):
        station.append(lis['number of station'][i])
        population.append(lis['population covered'][i])
    
    plt.plot(station,population, 'o')
    return
    

# Instructions:

# SEND ME A FOLDER CALLED 'name_of_your_city'

# Put inside this folder the following things:

## 1) The python file downloaded from this notebook after you will have modified the notebook as written below. 
## Call the python file(.py) 'name_of_your_city'

## 2) A folder called 'city_shape' with all the shapefiles of your final sorfuce. Remember, the final sourface, not the whole city.

## 3) An emty folder named 'station'


# DO NOT RUN THE ALGORITHM! 



## Remember!

### THE SHAPE FILE OF THE FINAL SURFACE, NOT THE WHOLE CITY!

# 

# Instruction on how to modify this notebook.


### first step: 

## In this line


In [34]:
city=gpd.read_file('buenos_aires_shape/manzanasSelectedForCluster.shp')

### instead of

### 'buenos_aires_shape/manzanasSelectedForCluster.shp'

### write

### 'city_shape' (like the folder you made) / the name of the shape file of your final sourface



#############

## In the following function change:

In [46]:
configuration2(city,300,10,'CENTROID_X','CENTROID_Y','popByManza')

## 'centroid_x' ---> the name of the column of the attribute table of your shapefile where you have your 'x' coordinate.

## 'centroid_y' ---> the name of the column of the attribute table of your shapefile where you have your 'y' coordinate.

## 'popByManza' ---> the name of the column of the attribute table of your shapefile where you have your 'Population' or the Area of your city.

######

# If you have no idea of what is going on, just send me stuff 